In [1]:
import sys
sys.path.append("..")
import numpy as np 
import torch
from relie.utils.so3_tools import so3_hat, so3_vee, so3_exp, so3_log
from relie.utils.se3_tools import se3_hat, se3_vee, se3_exp, se3_log, se3_inv

In [2]:
def det_jac_so3(x):
    theta = x.norm(2,-1)
    return 2*(1 - torch.cos(theta))/(theta**2)

def so3_inv(el):
    return el.transpose(-2,-1)
    

In [3]:
def compute_approx_jacobian(points, vee, exp, log, inv, eps = 0.01):
    points = points.unsqueeze(-2)
    dim = points.shape[-1]
    basis = torch.eye(dim, dtype=torch.float64).unsqueeze(0)
    group_delta = exp(basis*eps + points)
    points_inv = inv(exp(points))
    normal_coord = vee(log(points_inv@group_delta))
    print(normal_coord)
    estimated_det_jac = np.linalg.det(normal_coord.numpy())/((eps)**dim)
    return torch.tensor(estimated_det_jac)

### SO(3)

In [4]:
def approximate_so3_jacobian(points, eps = 0.01):
    return compute_approx_jacobian(points, so3_vee, so3_exp, so3_log, so3_inv, eps)

In [5]:
dim = 3
eps = 0.005

center = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
estimated_det_jac = approximate_so3_jacobian(center, eps=eps)
print(estimated_det_jac - det_jac_so3(center))

tensor([[[[ 0.0000, -0.0023,  0.0010],
          [ 0.0023,  0.0000, -0.0040],
          [-0.0010,  0.0040,  0.0000]],

         [[ 0.0000, -0.0023,  0.0027],
          [ 0.0023,  0.0000,  0.0025],
          [-0.0027, -0.0025,  0.0000]],

         [[ 0.0000, -0.0031, -0.0032],
          [ 0.0031,  0.0000,  0.0002],
          [ 0.0032, -0.0002,  0.0000]]],


        [[[ 0.0000, -0.0025, -0.0012],
          [ 0.0025,  0.0000, -0.0037],
          [ 0.0012,  0.0037,  0.0000]],

         [[ 0.0000, -0.0001,  0.0047],
          [ 0.0001,  0.0000, -0.0015],
          [-0.0047,  0.0015,  0.0000]],

         [[ 0.0000, -0.0040,  0.0010],
          [ 0.0040,  0.0000,  0.0023],
          [-0.0010, -0.0023,  0.0000]]],


        [[[ 0.0000, -0.0005, -0.0011],
          [ 0.0005,  0.0000, -0.0048],
          [ 0.0011,  0.0048,  0.0000]],

         [[ 0.0000,  0.0005,  0.0048],
          [-0.0005,  0.0000, -0.0012],
          [-0.0048,  0.0012,  0.0000]],

         [[ 0.0000, -0.0049,  0.0007],
     

### SE(3)

In [6]:
def approximate_se3_jacobian(points, eps = 0.01):
    return compute_approx_jacobian(points, se3_vee, se3_exp, se3_log, se3_inv, eps)

In [7]:
dim = 6
eps = 1

center = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
estimated_det_jac = approximate_se3_jacobian(center, eps=eps)

tensor([[[[ 0.0000,  0.0680, -0.1433],
          [-0.0680,  0.0000, -0.8231],
          [ 0.1433,  0.8231,  0.0000]],

         [[ 0.0000, -0.1104,  0.8211],
          [ 0.1104,  0.0000, -0.1254],
          [-0.8211,  0.1254,  0.0000]],

         [[ 0.0000, -0.8302, -0.1137],
          [ 0.8302,  0.0000, -0.0507],
          [ 0.1137,  0.0507,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000, -0.0057,  0.4747],
          [ 0.0057,  0.0000, -0.6452],
          [-0.4747,  0.6452,  0.0000]],

         [[ 0.0000,  0.5677,  0.5147],
          [-0.5677,  0.0000,  0.0929],
          [-0.5147, -0.0929,  0.0000]],

         [[ 0.0000, -0.5309,  0.4065],
       

/home/luca/anaconda2/envs/ds/lib/python3.5/site-packages/numpy/linalg/linalg.py:2010: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [8]:
theta = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
((1 - theta*torch.sin(theta))/(2-2*torch.cos(theta)))/theta**2

tensor([[ 4.8630e+00,  1.1728e+00,  4.2167e+01, -1.1316e-01,  8.6982e+00,
          3.5733e+00],
        [ 2.0757e+02, -1.1882e-01,  9.8925e-01, -5.0719e-02,  2.2513e-01,
          5.8712e+00],
        [ 1.2875e+02,  2.1224e+05, -4.9709e-02, -2.0581e-02,  1.5394e+01,
          5.6130e-01],
        [ 2.6940e+00,  1.2262e+00,  6.0627e-01,  2.4367e+01,  4.2098e+02,
          5.6445e-01],
        [ 5.5424e-01,  4.4701e+02,  1.3722e+02, -1.1583e-01,  3.6797e+00,
          1.3471e+08],
        [ 7.1742e-01,  4.5744e+00, -1.1850e-01, -7.5118e-02, -5.9573e-02,
          4.4602e+07],
        [-2.9001e-02, -1.1411e-01,  2.4626e+01,  8.5438e-02,  4.3831e+02,
          6.8869e+00],
        [-4.3417e-02,  2.9700e+02,  2.1250e+00,  6.3076e-01, -7.2156e-02,
          2.2267e+00],
        [ 9.0160e+01,  1.6445e+02,  3.1792e+00,  5.0561e+02, -1.0318e-01,
          3.2766e+02],
        [ 7.0042e+03, -1.0098e-01,  1.3022e+00,  2.5273e+00, -1.1861e-01,
         -9.2506e-02]], dtype=torch.float64)

In [9]:
x = so3_log(torch.eye(3, dtype = torch.float64).unsqueeze(0)) 
x

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], dtype=torch.float64)


tensor([[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]]], dtype=torch.float64)

In [10]:
mask_nan = x != x

In [11]:
x[mask_nan] = 0

In [12]:
x

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], dtype=torch.float64)